In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from typing import List, Optional
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

# --------------- Config ---------------
DATE = "20251106"
PRED_DATA_PATH = f"data/inference/{DATE}/inference_data.csv"  # your input dataframe (one row per game)
MODEL_DIR      = "./models/cbb_models_fast"           # where model_{target}.joblib live
OUT_CSV        = f"data/predictions/{DATE}/preds.csv"            # optional: will be created
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

TARGETS = [
    "home_1h", "away_1h", "home_2h", "away_2h",
    "home_score", "away_score", "home_margin", "away_margin"
]

TARGETS = [
    "home_1h", "away_1h", "home_2h", "away_2h",
    "home_score", "away_score", "home_margin", "away_margin"
]

PASSTHRU_COLS = ["game_id","game_id_y","date","date_utc","time_utc","home","away"]
NONNEG_TARGETS = {"home_1h","away_1h","home_2h","away_2h","home_score","away_score"}

def choose_date_column(df: pd.DataFrame) -> Optional[str]:
    if "date" in df.columns: return "date"
    if "date_utc" in df.columns: return "date_utc"
    return None

def _align_X_for_model(raw: pd.DataFrame, mdl) -> pd.DataFrame:
    """
    Build model input with the exact features LightGBM expects:
      • read expected names from Booster
      • create missing as NaN
      • convert any datetime columns to epoch seconds (float)
      • cast to float32
    """
    # Always use Booster names if available
    feat_list = None
    if hasattr(mdl, "_Booster") and mdl._Booster is not None:
        try:
            feat_list = list(mdl._Booster.feature_name())
        except Exception:
            pass
    if feat_list is None:
        feat_list = getattr(mdl, "feature_name_", None)

    if not feat_list:  # rare fallback
        cols = [c for c in raw.columns if is_numeric_dtype(raw[c])]
        return raw[cols].astype(np.float32)

    # Ensure all expected features exist
    for c in feat_list:
        if c not in raw.columns:
            raw[c] = np.nan

    # Convert datetime features to epoch seconds
    for c in feat_list:
        s = raw[c]
        if is_datetime64_any_dtype(s):
            s_dt = pd.to_datetime(s, errors="coerce")
            # convert to ns → float seconds; keep NaT as NaN
            vals = s_dt.view("int64").astype("float64")
            vals[s_dt.isna().values] = np.nan
            raw[c] = vals / 1e9  # seconds; matches “numeric” expectation

    # Keep exactly expected features, ordered, as float32
    X = raw[feat_list].astype(np.float32)
    return X

def run_inference():
    raw = pd.read_csv(PRED_DATA_PATH)

    # parse dates for display only
    for c in ("date","date_utc"):
        if c in raw.columns:
            raw[c] = pd.to_datetime(raw[c], errors="coerce")

    if "season" not in raw.columns:
        # use the season your data belongs to; constant is fine for future games
        raw["season"] = 2026
    # passthrough columns
    keep_cols = [c for c in PASSTHRU_COLS if c in raw.columns]
    id_col = "game_id" if "game_id" in keep_cols else ("game_id_y" if "game_id_y" in keep_cols else None)
    date_col = choose_date_column(raw)

    base_out_cols = []
    if id_col: base_out_cols.append(id_col)
    if date_col: base_out_cols.append(date_col)
    if "home" in raw.columns: base_out_cols.append("home")
    if "away" in raw.columns: base_out_cols.append("away")
    out = raw[base_out_cols].copy()
    out['season'] = 2026

    # predictions per target
    for tgt in TARGETS:
        path = os.path.join(MODEL_DIR, f"model_{tgt}.joblib")
        if not os.path.exists(path):
            out[f"pred_{tgt}"] = np.nan
            continue

        mdl = joblib.load(path)
        X = _align_X_for_model(raw, mdl)  # <-- guarantees numeric float32 only
        preds = mdl.predict(X)
        if tgt in NONNEG_TARGETS:
            preds = np.maximum(preds, 0.0)
        out[f"pred_{tgt}"] = preds.astype(np.float32)

    # tidy columns
    if "game_id_y" in out.columns and "game_id" not in out.columns:
        out = out.rename(columns={"game_id_y": "game_id"})

    pred_cols = [c for c in out.columns if c.startswith("pred_")]
    out = out[ [c for c in ["game_id", date_col, "home", "away"] if c in out.columns] + sorted(pred_cols) ]

    os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
    out['date'] = DATE
    out.to_csv(OUT_CSV, index=False)
    print(f"Saved predictions → {OUT_CSV}")
    return out

In [2]:
run_inference()

Saved predictions → data/predictions/20251106/preds.csv


,game_id,date,home,away,pred_away_1h,pred_away_2h,pred_away_margin,pred_away_score,pred_home_1h,pred_home_2h,pred_home_margin,pred_home_score
0,401822339.0,20251106,Saint Louis,Chicago St.,28.895231,34.362720,-28.059990,55.204540,42.685253,42.363689,28.059990,88.257195
1,401824031.0,20251106,Texas A&M,Texas Southern,23.206959,26.506794,-32.989960,49.226727,49.921566,42.886604,32.989960,86.885155
2,401812350.0,20251106,IU Indy,LIU,32.111900,35.551277,-11.673063,69.948540,38.576946,41.675705,11.673063,85.092606
3,NaN,20251106,Quinnipiac,Central Connecticut,34.522297,36.610008,-8.876779,71.512192,32.890240,35.117908,8.876779,73.682983
4,NaN,20251106,Florida,North Florida,30.700253,33.595478,-7.121861,72.827301,38.163067,38.383541,7.121861,75.498024
5,401820209.0,20251106,Loyola Chicago,Mercyhurst,32.238209,33.459732,-18.153334,61.653416,39.485786,35.240829,18.153334,79.022522
6,401829471.0,20251106,Auburn,Merrimack,29.345182,35.970520,-5.707523,66.727814,38.018688,41.642029,5.707523,76.920059
7,NaN,20251106,Boston College,The Citadel,33.330753,29.176563,-9.563465,61.150234,34.784908,34.623917,9.563465,71.889603
8,401812379.0,20251106,TCU,Saint Francis,33.087845,35.123657,-5.588025,69.496338,37.265602,41.826694,5.588025,82.774170
9,NaN,20251106,SMU,Texas A&M Corpus Chris,29.054459,32.827526,-15.052833,64.418709,37.627499,38.784416,15.052833,83.165901
